In [1]:
!pip install pandas
!pip install pinecone
!pip install langchain_google_genai
!pip install python-dotenv

  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.3/524.3 kB 1.6 MB/s eta 0:00:00-:--:--
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully uninstalled packaging-23.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [pinecone]4/5 [pinecone]plugin-assistant]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.1.23 requires packaging<24.0,>=23.2, but you have packaging 24.2 which is incompatible.
streamlit 1.28.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
  Attempting uninstall: packaging
    Found existing ins

In [7]:
from pinecone import Pinecone, ServerlessSpec
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import pandas as pd
from time import time
import dotenv
dotenv.load_dotenv()

True

In [3]:
if "GOOGLE_API_KEY" not in os.environ:
    print("Please set the GOOGLE_API_KEY environment variable.")
   

In [9]:
if "GOOGLE_CLOUD_PROJECT" not in os.environ or "GOOGLE_CLOUD_REGION" not in os.environ:
    print("Please set GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION environment variables for Vertex AI.")
    # For quick testing, you can uncomment and replace:
    # os.environ["GOOGLE_CLOUD_PROJECT"] = "your-gcp-project-id" # e.g., "my-project-12345"
    # os.environ["GOOGLE_CLOUD_REGION"] = "us-central1" # e.g., "us-central1"
    exit("Vertex AI project/region not set. Please set environment variables.")

In [13]:
dotenv.load_dotenv()

True

In [20]:

pinecone_api_key = "pcsk_5S2cpy_41jpruFsm5TPvGMCWZEBuE5RTkM8bT4XoqMBW5qJjKrsfjC5brDiPCH9GFTTqea"

In [21]:
pc = Pinecone(api_key=pinecone_api_key)

In [22]:


client = GoogleGenerativeAIEmbeddings(model="models/embedding-001") 

## Try out embeddings

In [26]:
embedding = client.embed_query("helloo there")

print(embedding)

[0.041495755, 0.014384685, -0.06567345, -0.03147624, 0.058659516, 0.008459238, 0.014193387, -0.027429312, -0.0069411243, 0.04771172, -0.02628475, 0.01685063, -0.04055326, 0.013362434, -0.019462084, -0.019906286, 0.012314993, 0.025338164, 0.030579597, -0.005077693, 0.029328683, 0.01872456, -0.0030511993, 0.007256962, 0.04212129, -0.022396091, 0.024133857, -0.026035402, -0.03512501, -0.008920383, -0.036745254, 0.010547488, -0.035346545, 0.016792215, 0.02460987, -0.060742043, 0.0054182885, 0.0133433165, -0.009213602, 0.0037820719, 0.012735728, -0.054769807, -0.03779811, 0.008245731, 0.01934423, 0.017024975, -0.044501595, 0.020249115, 0.016490737, -0.0474477, 0.04301519, 0.023565589, 0.07127201, -0.034877025, -0.011847475, -0.038979042, 0.061894834, 0.004014004, -0.020040235, 0.023549715, -0.0054484955, 0.008607946, 0.044222, 0.0058555645, -0.036254335, -0.066397846, -0.05777415, 0.019632012, 0.050096765, -0.005459263, -0.010499365, -0.061314814, 0.031477872, -0.0009826925, 0.009121051, -0

In [28]:
len(embedding)

768

## Wrangle dataset

In [7]:
df=pd.read_json('products/products.jsonl',lines=True)

In [8]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [11]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [12]:
df['text'].head()

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [26]:
texts = df['text'].tolist()

In [27]:
with open('products/Merry\'s_way_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [28]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

## Generate Embeddings

In [ ]:
output = client.embed_query(texts)

In [36]:
embeddings = output

## Push data to database

In [ ]:
index_name = "coffeeshop"

pc.create_index(
    name=index_name,
    dimension=768, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [ ]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

In [36]:
index_name = "coffeeshop"

In [37]:
index = pc.Index(index_name)

## Get Closest documents

In [34]:
text="Is Cappuccino lactose-free?"

In [35]:
output = client.embed_query(text)
embeding = output

In [38]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Cappuccino',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
              'score': 0.787013888,
              'values': []},
             {'id': 'Latte',
              'metadata': {'text': 'Latte : Smooth and creamy, our latte '
                                   'combines rich espresso with velvety '
                 